# ETL Exploration

### Notebook Set Up

In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt

In [113]:
import os

# Muestra la ruta actual de tu notebook
os.getcwd()


'/Users/nicoletondu/Desktop/data-science-thesis-2025'

### Read the data

In [116]:
import pandas as pd

# Paths correctos desde tu working directory actual
path_2023 = "data/raw-data/eda_data/station_passenger_counts_2023.csv"

# Lectura de los dos CSV
passengers_df = pd.read_csv(path_2023)

# Chequeo rápido
print("Shape:", passengers_df.shape)
passengers_df.head()

Shape: (156491, 5)


,TravelDate,DayOfWeek,Station,EntryTapCount,ExitTapCount
0,20230101,Sunday,Abbey Road DLR,360,322
1,20230101,Sunday,Abbey Wood,4375,4382
2,20230101,Sunday,Acton Central,779,709
3,20230101,Sunday,Acton Main Line,1087,967
4,20230101,Sunday,Acton Town,2608,3059


In [118]:
# Path relativo para la tabla de estaciones
stations_path = "data/processed/stations_dimension_table.csv"

# Lectura de la tabla de estaciones
stations_df = pd.read_csv(stations_path)

# Chequeo rápido
print("Shape:", df_stations.shape)
stations_df.head()

Shape: (946, 15)


,StopAreaNaptanCode,Mode,Line,UniqueId,Name,FareZones,HubNaptanCode,Wifi,AirportInterchange,BlueBadgeCarParking,BlueBadgeCarParkSpaces,StationUniqueId,IsAccessible,IsFeeCharged,Id
0,910GABWD,nationalRail,national-rail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,910GABWD,elizabeth-line,elizabeth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,910GACTNCTL,overground,mildmay,910GACTNCTL,Acton Central,3,NaN,True,NaN,False,NaN,NaN,NaN,NaN,NaN
3,910GACTONML,elizabeth-line,elizabeth,910GACTONML,Acton Main Line,3,NaN,False,NaN,False,NaN,910GACTONML,True,False,1.0
4,910GANERLEY,overground,windrush,910GANERLEY,Anerley,4,NaN,True,NaN,False,NaN,NaN,NaN,NaN,NaN


In [120]:
# Path relativo para la tabla de estaciones
events_path = "data/processed/event_sessions_2023.csv"

# Lectura de la tabla de estaciones
events_df = pd.read_csv(events_path)

# Chequeo rápido
print("Shape:", events_df.shape)
events_df.head()

Shape: (230, 5)


,date,event_type,event_name,affected_lines,expected_attendance
0,2023-01-02,Festivo,New Year’s Day (sustituto),General,NaN
1,2023-02-14,Concierto masivo,Chris Brown - Under the Influence Tour,Jubilee,NaN
2,2023-02-15,Concierto masivo,Chris Brown - Under the Influence Tour,Jubilee,NaN
3,2023-02-19,Concierto masivo,Chris Brown - Under the Influence Tour,Jubilee,NaN
4,2023-02-20,Concierto masivo,Chris Brown - Under the Influence Tour,Jubilee,NaN


In [122]:
### Data Cleaning and Transformation

# First, let's examine the current column names of both dataframes
print("PASSENGERS_DF COLUMNS:")
print(passengers_df.columns.tolist())
print("\nPASSENGERS_DF SAMPLE:")
print(passengers_df.head())
print("\nPASSENGERS_DF DATA TYPES:")
print(passengers_df.dtypes)

print("\n" + "="*50 + "\n")

print("STATIONS_DF COLUMNS:")
print(stations_df.columns.tolist())
print("\nSTATIONS_DF SAMPLE:")
print(stations_df.head())
print("\nSTATIONS_DF DATA TYPES:")
print(stations_df.dtypes)

PASSENGERS_DF COLUMNS:
['TravelDate', 'DayOfWeek', 'Station', 'EntryTapCount', 'ExitTapCount']

PASSENGERS_DF SAMPLE:
   TravelDate DayOfWeek          Station  EntryTapCount  ExitTapCount
0    20230101    Sunday   Abbey Road DLR            360           322
1    20230101    Sunday       Abbey Wood           4375          4382
2    20230101    Sunday    Acton Central            779           709
3    20230101    Sunday  Acton Main Line           1087           967
4    20230101    Sunday       Acton Town           2608          3059

PASSENGERS_DF DATA TYPES:
TravelDate        int64
DayOfWeek        object
Station          object
EntryTapCount     int64
ExitTapCount      int64
dtype: object


STATIONS_DF COLUMNS:
['StopAreaNaptanCode', 'Mode', 'Line', 'UniqueId', 'Name', 'FareZones', 'HubNaptanCode', 'Wifi', 'AirportInterchange', 'BlueBadgeCarParking', 'BlueBadgeCarParkSpaces', 'StationUniqueId', 'IsAccessible', 'IsFeeCharged', 'Id']

STATIONS_DF SAMPLE:
  StopAreaNaptanCode            

In [124]:
# 1. CLEAN PASSENGERS_DF
print("CLEANING PASSENGERS_DF...")

# Create a copy for cleaning
passengers_clean = passengers_df.copy()

# 1.1 Rename columns to lowercase with underscores
passengers_clean.columns = passengers_clean.columns.str.lower().str.replace(' ', '_')

# 1.2 Convert TravelDate to datetime (fix the date parsing issue)
# The TravelDate is in YYYYMMDD format as integers, so we need to convert to string first
passengers_clean['traveldate'] = pd.to_datetime(passengers_clean['traveldate'].astype(str), format='%Y%m%d')

# 1.3 Rename Station to station_name
passengers_clean = passengers_clean.rename(columns={'station': 'station_name'})

print("PASSENGERS_DF CLEANED COLUMNS:")
print(passengers_clean.columns.tolist())
print("\nPASSENGERS_DF CLEANED SAMPLE:")
print(passengers_clean.head())
print("\nPASSENGERS_DF CLEANED DATA TYPES:")
print(passengers_clean.dtypes)
print("\nDATE RANGE CHECK:")
print(f"Date range: {passengers_clean['traveldate'].min()} to {passengers_clean['traveldate'].max()}")

CLEANING PASSENGERS_DF...
PASSENGERS_DF CLEANED COLUMNS:
['traveldate', 'dayofweek', 'station_name', 'entrytapcount', 'exittapcount']

PASSENGERS_DF CLEANED SAMPLE:
  traveldate dayofweek     station_name  entrytapcount  exittapcount
0 2023-01-01    Sunday   Abbey Road DLR            360           322
1 2023-01-01    Sunday       Abbey Wood           4375          4382
2 2023-01-01    Sunday    Acton Central            779           709
3 2023-01-01    Sunday  Acton Main Line           1087           967
4 2023-01-01    Sunday       Acton Town           2608          3059

PASSENGERS_DF CLEANED DATA TYPES:
traveldate       datetime64[ns]
dayofweek                object
station_name             object
entrytapcount             int64
exittapcount              int64
dtype: object

DATE RANGE CHECK:
Date range: 2023-01-01 00:00:00 to 2023-12-31 00:00:00


In [126]:
# 2. CLEAN STATIONS_DF
print("CLEANING STATIONS_DF...")

# Create a copy for cleaning
stations_clean = stations_df.copy()

# 2.1 Rename columns to lowercase with underscores
stations_clean.columns = stations_clean.columns.str.lower().str.replace(' ', '_')

# 2.2 Remove StopAreaNaptanCode and StationUniqueId columns
columns_to_remove = ['stopareanaptancode', 'stationuniqueid']
stations_clean = stations_clean.drop(columns=columns_to_remove, errors='ignore')

# 2.3 Rename UniqueId to station_uid
stations_clean = stations_clean.rename(columns={'uniqueid': 'station_uid'})

# 2.4 Rename Name to station_name
stations_clean = stations_clean.rename(columns={'name': 'station_name'})

CLEANING STATIONS_DF...


In [128]:
# 3. UPDATE ORIGINAL DATAFRAMES AND SUMMARY
print("UPDATING ORIGINAL DATAFRAMES...")

# Update the original dataframes with cleaned versions
passengers_df = passengers_clean.copy()
stations_df = stations_clean.copy()

print("="*60)
print("DATA CLEANING SUMMARY")
print("="*60)

print(f"\nPASSENGERS_DF:")
print(f"  - Shape: {passengers_df.shape}")
print(f"  - Columns: {list(passengers_df.columns)}")
print(f"  - Date range: {passengers_df['traveldate'].min().strftime('%Y-%m-%d')} to {passengers_df['traveldate'].max().strftime('%Y-%m-%d')}")
print(f"  - Total days: {(passengers_df['traveldate'].max() - passengers_df['traveldate'].min()).days + 1}")

print(f"\nSTATIONS_DF:")
print(f"  - Shape: {stations_df.shape}")
print(f"  - Columns: {list(stations_df.columns)}")
print(f"  - Unique stations: {stations_df['station_name'].nunique()}")

print(f"\nEVENTS_DF:")
print(f"  - Shape: {events_df.shape}")
print(f"  - Columns: {list(events_df.columns)}")

print("\n" + "="*60)
print("ALL DATAFRAMES ARE NOW CLEANED AND READY FOR ML!")
print("="*60)

UPDATING ORIGINAL DATAFRAMES...
DATA CLEANING SUMMARY

PASSENGERS_DF:
  - Shape: (156491, 5)
  - Columns: ['traveldate', 'dayofweek', 'station_name', 'entrytapcount', 'exittapcount']
  - Date range: 2023-01-01 to 2023-12-31
  - Total days: 365

STATIONS_DF:
  - Shape: (946, 13)
  - Columns: ['mode', 'line', 'station_uid', 'station_name', 'farezones', 'hubnaptancode', 'wifi', 'airportinterchange', 'bluebadgecarparking', 'bluebadgecarparkspaces', 'isaccessible', 'isfeecharged', 'id']
  - Unique stations: 414

EVENTS_DF:
  - Shape: (230, 5)
  - Columns: ['date', 'event_type', 'event_name', 'affected_lines', 'expected_attendance']

ALL DATAFRAMES ARE NOW CLEANED AND READY FOR ML!


In [130]:
# 4. JOIN PASSENGERS_DF AND STATIONS_DF BY STATION NAME
print("JOINING PASSENGERS_DF AND STATIONS_DF...")

# Check station name compatibility before joining
print("STATION NAME COMPATIBILITY CHECK:")
print(f"Unique stations in passengers_df: {passengers_df['station_name'].nunique()}")
print(f"Unique stations in stations_df: {stations_df['station_name'].nunique()}")

# Check for any missing station names
passengers_stations = set(passengers_df['station_name'].unique())
stations_stations = set(stations_df['station_name'].unique())

missing_in_stations = passengers_stations - stations_stations
missing_in_passengers = stations_stations - passengers_stations

print(f"\nStations in passengers_df but not in stations_df: {len(missing_in_stations)}")
if len(missing_in_stations) > 0:
    print("Missing stations:", list(missing_in_stations)[:10])  # Show first 10

print(f"\nStations in stations_df but not in passengers_df: {len(missing_in_passengers)}")
if len(missing_in_passengers) > 0:
    print("Extra stations:", list(missing_in_passengers)[:10])  # Show first 10

# Perform the join
print("\n" + "="*50)
print("PERFORMING INNER JOIN...")
passengers_with_stations = passengers_df.merge(
    stations_df, 
    on='station_name', 
    how='inner'
)

print(f"Original passengers_df shape: {passengers_df.shape}")
print(f"Joined dataframe shape: {passengers_with_stations.shape}")
print(f"Rows lost in join: {passengers_df.shape[0] - passengers_with_stations.shape[0]}")

print("\nJOINED DATAFRAME COLUMNS:")
print(passengers_with_stations.columns.tolist())

print("\nJOINED DATAFRAME SAMPLE:")
print(passengers_with_stations.head())


JOINING PASSENGERS_DF AND STATIONS_DF...
STATION NAME COMPATIBILITY CHECK:
Unique stations in passengers_df: 434
Unique stations in stations_df: 414

Stations in passengers_df but not in stations_df: 119
Missing stations: ['Harlesden', 'Shadwell LO', 'Hammersmith D&P', 'W Silvertown', 'Westminster', 'Gunnersbury', 'Walthamstow Central', 'Rickmansworth', 'Imperial Wharf', 'Kew Gardens']

Stations in stations_df but not in passengers_df: 100
Extra stations: ['Edgware Road', 'Woolwich', 'New Barnet', 'IFS Cloud Royal Docks', "Regent's Park", 'George Street', 'Addington Village', 'Swanley', 'Deptford', 'Lloyd Park']

PERFORMING INNER JOIN...
Original passengers_df shape: (156491, 5)
Joined dataframe shape: (212853, 17)
Rows lost in join: -56362

JOINED DATAFRAME COLUMNS:
['traveldate', 'dayofweek', 'station_name', 'entrytapcount', 'exittapcount', 'mode', 'line', 'station_uid', 'farezones', 'hubnaptancode', 'wifi', 'airportinterchange', 'bluebadgecarparking', 'bluebadgecarparkspaces', 'isac

In [132]:
# 5. UPDATE PASSENGERS_DF WITH STATION INFORMATION
print("UPDATING PASSENGERS_DF WITH STATION INFORMATION...")

# Update the original passengers_df with the joined data
passengers_df = passengers_with_stations.copy()

print("="*60)
print("JOIN SUMMARY")
print("="*60)

print(f"\nFINAL PASSENGERS_DF:")
print(f"  - Shape: {passengers_df.shape}")
print(f"  - Columns: {list(passengers_df.columns)}")
print(f"  - Date range: {passengers_df['traveldate'].min().strftime('%Y-%m-%d')} to {passengers_df['traveldate'].max().strftime('%Y-%m-%d')}")
print(f"  - Unique stations: {passengers_df['station_name'].nunique()}")
print(f"  - Unique lines: {passengers_df['line'].nunique()}")
print(f"  - Unique modes: {passengers_df['mode'].nunique()}")

print(f"\nSTATION INFORMATION ADDED:")
print(f"  - Mode: {passengers_df['mode'].value_counts().head()}")
print(f"  - Line: {passengers_df['line'].value_counts().head()}")
print(f"  - Fare zones: {passengers_df['farezones'].value_counts().head()}")

print("\n" + "="*60)
print("PASSENGERS_DF NOW CONTAINS STATION INFORMATION!")
print("="*60)


UPDATING PASSENGERS_DF WITH STATION INFORMATION...
JOIN SUMMARY

FINAL PASSENGERS_DF:
  - Shape: (212853, 17)
  - Columns: ['traveldate', 'dayofweek', 'station_name', 'entrytapcount', 'exittapcount', 'mode', 'line', 'station_uid', 'farezones', 'hubnaptancode', 'wifi', 'airportinterchange', 'bluebadgecarparking', 'bluebadgecarparkspaces', 'isaccessible', 'isfeecharged', 'id']
  - Date range: 2023-01-01 to 2023-12-31
  - Unique stations: 315
  - Unique lines: 19
  - Unique modes: 5

STATION INFORMATION ADDED:
  - Mode: mode
tube              134264
overground         32802
elizabeth-line     18679
nationalRail       16601
dlr                10507
Name: count, dtype: int64
  - Line: line
piccadilly      24935
central         21710
district        19911
elizabeth       18679
metropolitan    17325
Name: count, dtype: int64
  - Fare zones: farezones
2    43330
4    33822
1    30844
3    25915
6    23726
Name: count, dtype: int64

PASSENGERS_DF NOW CONTAINS STATION INFORMATION!


In [138]:
# 6. JOIN WITH EVENTS TABLE (BY DATE AND LINE)
print("JOINING WITH EVENTS TABLE...")

# Ensure proper date types and create date-only columns
passengers_df = passengers_df.copy()
passengers_df['date'] = passengers_df['traveldate'].dt.date

events_df = events_df.copy()
events_df['date'] = pd.to_datetime(events_df['date']).dt.date

# Normalize line names to improve matching (remove non-letters, lowercase)
passengers_df['line_norm'] = (
    passengers_df['line'].astype(str).str.lower().str.replace(r'[^a-z]', '', regex=True)
)

events_df['affected_line_norm'] = (
    events_df['affected_lines'].astype(str).str.lower().str.replace(r'[^a-z]', '', regex=True)
)

# Perform left join to keep all passenger rows and attach events when present
passengers_events = passengers_df.merge(
    events_df[['date', 'affected_line_norm', 'event_type', 'event_name', 'expected_attendance']],
    left_on=['date', 'line_norm'],
    right_on=['date', 'affected_line_norm'],
    how='left'
)

# Cleanup helper columns
passengers_events = passengers_events.drop(columns=['affected_line_norm'])

# Quick summary
num_matched = passengers_events['event_name'].notna().sum()
print(f"Total rows: {len(passengers_events)}")
print(f"Rows with an event match: {num_matched}")
print("Event type distribution (matched only):")
print(passengers_events.loc[passengers_events['event_name'].notna(), 'event_type'].value_counts())

print("\nSAMPLE OF JOINED DATA:")
print(passengers_events[['date','station_name','line','entrytapcount','exittapcount','event_type','event_name','expected_attendance']].head(10))


JOINING WITH EVENTS TABLE...
Total rows: 213134
Rows with an event match: 7770
Event type distribution (matched only):
event_type
Concierto masivo                 5112
Evento deportivo                  851
Festivo                           550
Evento cultural                   539
Festival/Evento al aire libre     395
Evento deportivo/espectáculo      199
Evento deportivo/benéfico         124
Name: count, dtype: int64

SAMPLE OF JOINED DATA:
         date     station_name          line  entrytapcount  exittapcount  \
0  2023-01-01    Acton Central       mildmay            779           709   
1  2023-01-01  Acton Main Line     elizabeth           1087           967   
2  2023-01-01       Acton Town      district           2608          3059   
3  2023-01-01       Acton Town      district           2608          3059   
4  2023-01-01       Acton Town      district           2608          3059   
5  2023-01-01       Acton Town    piccadilly           2608          3059   
6  2023-01-01  

In [142]:
passengers_df.head()

,traveldate,dayofweek,station_name,entrytapcount,exittapcount,mode,line,station_uid,farezones,hubnaptancode,wifi,airportinterchange,bluebadgecarparking,bluebadgecarparkspaces,isaccessible,isfeecharged,id,date,line_norm
0,2023-01-01,Sunday,Acton Central,779,709,overground,mildmay,910GACTNCTL,3,NaN,True,NaN,False,NaN,NaN,NaN,NaN,2023-01-01,mildmay
1,2023-01-01,Sunday,Acton Main Line,1087,967,elizabeth-line,elizabeth,910GACTONML,3,NaN,False,NaN,False,NaN,True,False,1.0,2023-01-01,elizabeth
2,2023-01-01,Sunday,Acton Town,2608,3059,tube,district,940GZZLUACT,3,NaN,True,NaN,False,NaN,False,False,1.0,2023-01-01,district
3,2023-01-01,Sunday,Acton Town,2608,3059,tube,district,940GZZLUACT,3,NaN,True,NaN,False,NaN,False,False,2.0,2023-01-01,district
4,2023-01-01,Sunday,Acton Town,2608,3059,tube,district,940GZZLUACT,3,NaN,True,NaN,False,NaN,True,False,3.0,2023-01-01,district


In [146]:
passengers_events.head()

,traveldate,dayofweek,station_name,entrytapcount,exittapcount,mode,line,station_uid,farezones,hubnaptancode,...,bluebadgecarparking,bluebadgecarparkspaces,isaccessible,isfeecharged,id,date,line_norm,event_type,event_name,expected_attendance
0,2023-01-01,Sunday,Acton Central,779,709,overground,mildmay,910GACTNCTL,3,NaN,...,False,NaN,NaN,NaN,NaN,2023-01-01,mildmay,NaN,NaN,NaN
1,2023-01-01,Sunday,Acton Main Line,1087,967,elizabeth-line,elizabeth,910GACTONML,3,NaN,...,False,NaN,True,False,1.0,2023-01-01,elizabeth,NaN,NaN,NaN
2,2023-01-01,Sunday,Acton Town,2608,3059,tube,district,940GZZLUACT,3,NaN,...,False,NaN,False,False,1.0,2023-01-01,district,NaN,NaN,NaN
3,2023-01-01,Sunday,Acton Town,2608,3059,tube,district,940GZZLUACT,3,NaN,...,False,NaN,False,False,2.0,2023-01-01,district,NaN,NaN,NaN
4,2023-01-01,Sunday,Acton Town,2608,3059,tube,district,940GZZLUACT,3,NaN,...,False,NaN,True,False,3.0,2023-01-01,district,NaN,NaN,NaN


In [ ]:
# 7. ENRICH WITH WEATHER AND MONTHLY PERFORMANCE DATA
print("ENRICHING WITH WEATHER AND MONTHLY PERFORMANCE DATA...")

# Make a working copy
passengers_events_enriched = passengers_events.copy()

# 7.1 Load and prepare weather (daily, citywide)
weather_path = "data/processed/london_weather_2023_clean.csv"
weather_df = pd.read_csv(weather_path)

# Normalize columns
weather_df.columns = weather_df.columns.str.strip().str.lower().str.replace(' ', '_')
# Parse DATE in YYYYMMDD format
weather_df['date'] = pd.to_datetime(weather_df['date'].astype(str), format='%Y%m%d').dt.date

# Keep useful weather features
weather_keep_cols = [
    'date', 'max_temp', 'min_temp', 'mean_temp', 'precipitation_amount',
    'relative_humidity', 'cloud_cover', 'sunshine_duration', 'sea_level_pressure'
]
weather_df = weather_df[weather_keep_cols]

# Merge on date only (weather is not line-specific)
passengers_events_enriched = passengers_events_enriched.merge(
    weather_df,
    on='date',
    how='left'
)

# 7.2 Load and prepare monthly tube performance (monthly, by line)
perf_path = "data/processed/monthly_tube_performance_2023.csv"
perf_df = pd.read_csv(perf_path)

# Normalize columns
perf_df.columns = perf_df.columns.str.strip().str.lower().str.replace(' ', '_')

# Standardize line names: map short codes and expand combined lines
perf_df['line_name'] = perf_df['line_name'].replace({
    'C&H': 'Circle Hammersmith & City'
})

# Expand combined "Circle Hammersmith & City" into two rows
mask_combined = perf_df['line_name'].astype(str).str.lower().str.contains('circle') & perf_df['line_name'].astype(str).str.lower().str.contains('hammersmith')
combined_rows = perf_df[mask_combined]

if not combined_rows.empty:
    dup1 = combined_rows.copy()
    dup1['line_name'] = 'Circle'
    dup2 = combined_rows.copy()
    dup2['line_name'] = 'Hammersmith & City'
    perf_df = pd.concat([perf_df[~mask_combined], dup1, dup2], ignore_index=True)

# Create normalized line key to match passengers' line_norm
perf_df['line_norm'] = perf_df['line_name'].astype(str).str.lower().str.replace(r'[^a-z]', '', regex=True)

# Compute year-month keys to match daily data
perf_df['year'] = perf_df['year'].astype(int)
perf_df['month'] = perf_df['month'].astype(int)

# Derive year and month from passengers' date
passengers_events_enriched['year'] = pd.to_datetime(passengers_events_enriched['date']).dt.year
passengers_events_enriched['month'] = pd.to_datetime(passengers_events_enriched['date']).dt.month

# Select performance metrics to merge
perf_keep_cols = [
    'year', 'month', 'line_norm',
    'service_operated_allweek_pct', 'service_operated_weekday_pct',
    'service_operated_weekend_pct', 'kilometres_operated'
]
perf_df = perf_df[perf_keep_cols]

# Merge monthly performance by (year, month, line_norm)
passengers_events_enriched = passengers_events_enriched.merge(
    perf_df,
    on=['year', 'month', 'line_norm'],
    how='left'
)

print("Done. Shapes:")
print("  passengers_events:", passengers_events.shape)
print("  weather_df:", weather_df.shape)
print("  perf_df:", perf_df.shape)
print("  passengers_events_enriched:", passengers_events_enriched.shape)

print("\nSample of enriched data:")
print(passengers_events_enriched[[
    'date','station_name','line','event_type','event_name','expected_attendance',
    'mean_temp','precipitation_amount','service_operated_allweek_pct'
]].head(10))


In [ ]:
# 8. UNIFIED ENRICHMENT PIPELINE
print("RUNNING UNIFIED ENRICHMENT PIPELINE...")

# 8.1 Load all source tables fresh
passengers_src = pd.read_csv("data/raw-data/eda_data/station_passenger_counts_2023.csv")
stations_src = pd.read_csv("data/processed/stations_dimension_table.csv")
events_src = pd.read_csv("data/processed/event_sessions_2023.csv")
weather_src = pd.read_csv("data/processed/london_weather_2023_clean.csv")
perf_src = pd.read_csv("data/processed/monthly_tube_performance_2023.csv")

# 8.2 Normalize schemas
# passengers
p = passengers_src.copy()
p.columns = p.columns.str.lower().str.replace(' ', '_')
p['traveldate'] = pd.to_datetime(p['traveldate'].astype(str), format='%Y%m%d')
p = p.rename(columns={'station': 'station_name'})

# stations
s = stations_src.copy()
s.columns = s.columns.str.lower().str.replace(' ', '_')
s = s.drop(columns=['stopareanaptancode', 'stationuniqueid'], errors='ignore')
s = s.rename(columns={'uniqueid': 'station_uid', 'name': 'station_name'})

# events
e = events_src.copy()
e.columns = e.columns.str.strip()
e['date'] = pd.to_datetime(e['date']).dt.date

# weather
w = weather_src.copy()
w.columns = w.columns.str.strip().str.lower().str.replace(' ', '_')
w['date'] = pd.to_datetime(w['date'].astype(str), format='%Y%m%d').dt.date
weather_keep = ['date','max_temp','min_temp','mean_temp','precipitation_amount','relative_humidity','cloud_cover','sunshine_duration','sea_level_pressure']
w = w[weather_keep]

# performance (monthly by line)
perf = perf_src.copy()
perf.columns = perf.columns.str.strip().str.lower().str.replace(' ', '_')
perf['line_name'] = perf['line_name'].replace({'C&H': 'Circle Hammersmith & City'})
mask_combined = perf['line_name'].astype(str).str.lower().str.contains('circle') & perf['line_name'].astype(str).str.lower().str.contains('hammersmith')
combined_rows = perf[mask_combined]
if not combined_rows.empty:
    dup1 = combined_rows.copy(); dup1['line_name'] = 'Circle'
    dup2 = combined_rows.copy(); dup2['line_name'] = 'Hammersmith & City'
    perf = pd.concat([perf[~mask_combined], dup1, dup2], ignore_index=True)
perf['line_norm'] = perf['line_name'].astype(str).str.lower().str.replace(r'[^a-z]', '', regex=True)
perf['year'] = perf['year'].astype(int)
perf['month'] = perf['month'].astype(int)
perf_keep = ['year','month','line_norm','service_operated_allweek_pct','service_operated_weekday_pct','service_operated_weekend_pct','kilometres_operated']
perf = perf[perf_keep]

# 8.3 Keys and helpers
# passengers station join
p_s = p.merge(s, on='station_name', how='inner')

# passengers + stations with events (by date + line)
p_s['date'] = p_s['traveldate'].dt.date
p_s['line_norm'] = p_s['line'].astype(str).str.lower().str.replace(r'[^a-z]', '', regex=True)

e['affected_line_norm'] = e['affected_lines'].astype(str).str.lower().str.replace(r'[^a-z]', '', regex=True)

ps_e = p_s.merge(
    e[['date','affected_line_norm','event_type','event_name','expected_attendance']],
    left_on=['date','line_norm'], right_on=['date','affected_line_norm'], how='left'
).drop(columns=['affected_line_norm'])

# add weather by date
ps_e_w = ps_e.merge(w, on='date', how='left')

# add performance by (year,month,line)
ps_e_w['year'] = pd.to_datetime(ps_e_w['date']).dt.year
ps_e_w['month'] = pd.to_datetime(ps_e_w['date']).dt.month
ps_e_w_perf = ps_e_w.merge(perf, on=['year','month','line_norm'], how='left')

# 8.4 Select and order columns
id_cols = ['date','dayofweek','station_name','station_uid','mode','line','farezones','hubnaptancode']
counts_cols = ['entrytapcount','exittapcount']
event_cols = ['event_type','event_name','expected_attendance']
weather_cols = ['max_temp','min_temp','mean_temp','precipitation_amount','relative_humidity','cloud_cover','sunshine_duration','sea_level_pressure']
perf_cols = ['service_operated_allweek_pct','service_operated_weekday_pct','service_operated_weekend_pct','kilometres_operated']
other_station_cols = ['wifi','airportinterchange','bluebadgecarparking','bluebadgecarparkspaces','isaccessible','isfeecharged','id']

ordered_cols = id_cols + counts_cols + event_cols + weather_cols + perf_cols + other_station_cols

# Keep only columns that exist
ordered_cols = [c for c in ordered_cols if c in ps_e_w_perf.columns]

# Drop traveldate if present
ps_e_w_perf = ps_e_w_perf.drop(columns=['traveldate'], errors='ignore')

passengers_enriched_2023 = ps_e_w_perf[ordered_cols].copy()

# 8.5 Save CSV
out_path = "data/processed/passengers_enriched_2023.csv"
passengers_enriched_2023.to_csv(out_path, index=False)

print("Pipeline complete. Final shape:", passengers_enriched_2023.shape)
print("Saved:", out_path)
print("Columns order:")
print(passengers_enriched_2023.columns.tolist())


RUNNING UNIFIED ENRICHMENT PIPELINE...
Pipeline complete. Final shape: (261028, 33)
Saved: data/processed/passengers_enriched_2023.csv
Columns order:
['traveldate', 'date', 'dayofweek', 'station_name', 'station_uid', 'mode', 'line', 'farezones', 'hubnaptancode', 'entrytapcount', 'exittapcount', 'event_type', 'event_name', 'expected_attendance', 'max_temp', 'min_temp', 'mean_temp', 'precipitation_amount', 'relative_humidity', 'cloud_cover', 'sunshine_duration', 'sea_level_pressure', 'service_operated_allweek_pct', 'service_operated_weekday_pct', 'service_operated_weekend_pct', 'kilometres_operated', 'wifi', 'airportinterchange', 'bluebadgecarparking', 'bluebadgecarparkspaces', 'isaccessible', 'isfeecharged', 'id']


### 9. Stepwise Enrichment Pipeline (Readable)
This section shows each step explicitly: load, normalize, join passengers+stations, add events, add weather, add performance, and finalize columns/saving. Each step prints a short summary.


In [ ]:
# 9.1 Load all sources
print("Loading sources...")
passengers_src = pd.read_csv("data/raw-data/eda_data/station_passenger_counts_2023.csv")
stations_src   = pd.read_csv("data/processed/stations_dimension_table.csv")
events_src     = pd.read_csv("data/processed/event_sessions_2023.csv")
weather_src    = pd.read_csv("data/processed/london_weather_2023_clean.csv")
perf_src       = pd.read_csv("data/processed/monthly_tube_performance_2023.csv")

for name, df in {
    'passengers_src': passengers_src,
    'stations_src': stations_src,
    'events_src': events_src,
    'weather_src': weather_src,
    'perf_src': perf_src,
}.items():
    print(f"{name}: {df.shape}")


In [ ]:
# 9.2 Normalize schemas and keys
print("Normalizing...")

# Passengers
p = passengers_src.copy()
p.columns = p.columns.str.lower().str.replace(' ', '_')
p['traveldate'] = pd.to_datetime(p['traveldate'].astype(str), format='%Y%m%d')
p = p.rename(columns={'station': 'station_name'})

# Stations
s = stations_src.copy()
s.columns = s.columns.str.lower().str.replace(' ', '_')
s = s.drop(columns=['stopareanaptancode', 'stationuniqueid'], errors='ignore')
s = s.rename(columns={'uniqueid': 'station_uid', 'name': 'station_name'})

# Events
e = events_src.copy()
e['date'] = pd.to_datetime(e['date']).dt.date

e['affected_line_norm'] = e['affected_lines'].astype(str).str.lower().str.replace(r'[^a-z]', '', regex=True)

# Weather
w = weather_src.copy()
w.columns = w.columns.str.strip().str.lower().str.replace(' ', '_')
w['date'] = pd.to_datetime(w['date'].astype(str), format='%Y%m%d').dt.date
w = w[['date','max_temp','min_temp','mean_temp','precipitation_amount','relative_humidity','cloud_cover','sunshine_duration','sea_level_pressure']]

# Performance (monthly by line)
perf = perf_src.copy()
perf.columns = perf.columns.str.strip().str.lower().str.replace(' ', '_')
perf['line_name'] = perf['line_name'].replace({'C&H': 'Circle Hammersmith & City'})
mask_combined = perf['line_name'].astype(str).str.lower().str.contains('circle') & perf['line_name'].astype(str).str.lower().str.contains('hammersmith')
combined_rows = perf[mask_combined]
if not combined_rows.empty:
    dup1 = combined_rows.copy(); dup1['line_name'] = 'Circle'
    dup2 = combined_rows.copy(); dup2['line_name'] = 'Hammersmith & City'
    perf = pd.concat([perf[~mask_combined], dup1, dup2], ignore_index=True)
perf['line_norm'] = perf['line_name'].astype(str).str.lower().str.replace(r'[^a-z]', '', regex=True)
perf['year'] = perf['year'].astype(int)
perf['month'] = perf['month'].astype(int)
perf = perf[['year','month','line_norm','service_operated_allweek_pct','service_operated_weekday_pct','service_operated_weekend_pct','kilometres_operated']]

print("Done.")


In [ ]:
# 9.3 Join passengers + stations (on station_name)
print("Join 1/4: passengers + stations...")

p_s = p.merge(s, on='station_name', how='inner')
print("passengers:", p.shape, "stations:", s.shape, "joined:", p_s.shape)
print("Columns added:", [c for c in p_s.columns if c not in p.columns])

# Prepare keys for next steps
p_s['date'] = p_s['traveldate'].dt.date
p_s['line_norm'] = p_s['line'].astype(str).str.lower().str.replace(r'[^a-z]', '', regex=True)

p_s.head()


In [ ]:
# 9.4 Join events (on date + line)
print("Join 2/4: add events...")

ps_e = p_s.merge(
    e[['date','affected_line_norm','event_type','event_name','expected_attendance']],
    left_on=['date','line_norm'], right_on=['date','affected_line_norm'], how='left'
).drop(columns=['affected_line_norm'])

matched = ps_e['event_name'].notna().sum()
print("rows:", ps_e.shape, "matched events:", matched)
ps_e.head()


In [ ]:
# 9.5 Join weather (on date)
print("Join 3/4: add weather...")

ps_e_w = ps_e.merge(w, on='date', how='left')
print("rows:", ps_e_w.shape)
ps_e_w.head()


In [ ]:
# 9.6 Join monthly performance (on year, month, line)
print("Join 4/4: add monthly performance...")

ps_e_w['year'] = pd.to_datetime(ps_e_w['date']).dt.year
ps_e_w['month'] = pd.to_datetime(ps_e_w['date']).dt.month

ps_e_w_perf = ps_e_w.merge(perf, on=['year','month','line_norm'], how='left')
print("rows:", ps_e_w_perf.shape)
ps_e_w_perf.head()


In [ ]:
# 9.7 Finalize columns, save, and preview
id_cols = ['date','dayofweek','station_name','station_uid','mode','line','farezones','hubnaptancode']
counts_cols = ['entrytapcount','exittapcount']
event_cols = ['event_type','event_name','expected_attendance']
weather_cols = ['max_temp','min_temp','mean_temp','precipitation_amount','relative_humidity','cloud_cover','sunshine_duration','sea_level_pressure']
perf_cols = ['service_operated_allweek_pct','service_operated_weekday_pct','service_operated_weekend_pct','kilometres_operated']
other_station_cols = ['wifi','airportinterchange','bluebadgecarparking','bluebadgecarparkspaces','isaccessible','isfeecharged','id']

# Drop traveldate if present
ps_e_w_perf = ps_e_w_perf.drop(columns=['traveldate'], errors='ignore')

ordered_cols = id_cols + counts_cols + event_cols + weather_cols + perf_cols + other_station_cols
ordered_cols = [c for c in ordered_cols if c in ps_e_w_perf.columns]

passengers_enriched_2023 = ps_e_w_perf[ordered_cols].copy()

out_path = "data/processed/passengers_enriched_2023.csv"
passengers_enriched_2023.to_csv(out_path, index=False)

print("Final shape:", passengers_enriched_2023.shape)
print("Saved:", out_path)

Final shape: (261028, 33)
Saved: data/processed/passengers_enriched_2023.csv


In [166]:
passengers_enriched_2023.head(10)

,traveldate,date,dayofweek,station_name,station_uid,mode,line,farezones,hubnaptancode,entrytapcount,...,service_operated_weekday_pct,service_operated_weekend_pct,kilometres_operated,wifi,airportinterchange,bluebadgecarparking,bluebadgecarparkspaces,isaccessible,isfeecharged,id
0,2023-01-01,2023-01-01,Sunday,Acton Central,910GACTNCTL,overground,mildmay,3,NaN,779,...,NaN,NaN,NaN,True,NaN,False,NaN,NaN,NaN,NaN
1,2023-01-01,2023-01-01,Sunday,Acton Main Line,910GACTONML,elizabeth-line,elizabeth,3,NaN,1087,...,NaN,NaN,NaN,False,NaN,False,NaN,True,False,1.0
2,2023-01-01,2023-01-01,Sunday,Acton Town,940GZZLUACT,tube,district,3,NaN,2608,...,93.0,80.8,165295.0,True,NaN,False,NaN,False,False,1.0
3,2023-01-01,2023-01-01,Sunday,Acton Town,940GZZLUACT,tube,district,3,NaN,2608,...,93.0,80.8,165295.0,True,NaN,False,NaN,False,False,2.0
4,2023-01-01,2023-01-01,Sunday,Acton Town,940GZZLUACT,tube,district,3,NaN,2608,...,93.0,80.8,165295.0,True,NaN,False,NaN,True,False,3.0
5,2023-01-01,2023-01-01,Sunday,Acton Town,940GZZLUACT,tube,piccadilly,3,NaN,2608,...,84.9,83.3,196661.0,True,NaN,False,NaN,False,False,1.0
6,2023-01-01,2023-01-01,Sunday,Acton Town,940GZZLUACT,tube,piccadilly,3,NaN,2608,...,84.9,83.3,196661.0,True,NaN,False,NaN,False,False,2.0
7,2023-01-01,2023-01-01,Sunday,Acton Town,940GZZLUACT,tube,piccadilly,3,NaN,2608,...,84.9,83.3,196661.0,True,NaN,False,NaN,True,False,3.0
8,2023-01-01,2023-01-01,Sunday,Aldgate,940GZZLUALD,tube,circle,1,NaN,6146,...,NaN,NaN,51443.0,True,NaN,False,NaN,NaN,NaN,NaN
9,2023-01-01,2023-01-01,Sunday,Aldgate,940GZZLUALD,tube,circle,1,NaN,6146,...,86.7,72.8,NaN,True,NaN,False,NaN,NaN,NaN,NaN
